# Age-Related Conditions Predictor

A machine learning solution using XGBoost for predicting age-related conditions from anonymized health data. The model performs binary classification with standardized features and optimized hyperparameters to generate probability-based predictions while maintaining patient privacy.

Key features:
- Binary classification
- Standardized preprocessing
- Automated deployment

Dataset: https://www.kaggle.com/competitions/icr-identify-age-related-conditions/data

Hugging Face: https://huggingface.co/spaces/alperugurcan/Age-Related-Conditions-Predictor

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import joblib

def train_and_predict():
    train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
    test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
    
    features = [col for col in train.columns if col not in ['Id', 'Class']]
    X_train = train[features].copy()  # Create explicit copy
    X_test = test[features].copy()    # Create explicit copy
    y_train = train['Class']
    
    X_train.loc[:, 'EJ'] = pd.Categorical(X_train['EJ']).codes
    X_test.loc[:, 'EJ'] = pd.Categorical(X_test['EJ']).codes
    
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=features)
    
    model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.01,
        subsample=0.8,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    preds = model.predict_proba(X_test)
    
    submission = pd.DataFrame({
        'Id': test['Id'],
        'class_0': preds[:, 0],
        'class_1': preds[:, 1]
    })
    
    submission.to_csv('submission.csv', index=False)
    joblib.dump(model, 'model.joblib')
    joblib.dump(scaler, 'scaler.joblib')

if __name__ == "__main__":
    train_and_predict()